In [7]:
# %% Cell 1: Install Required Package (if needed)
import subprocess
import sys

try:
    from google.cloud import storage
    print("✅ google-cloud-storage already installed")
except ImportError:
    print("Installing google-cloud-storage...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "google-cloud-storage"])
    print("✅ Installation complete. Please restart kernel if needed.")

✅ google-cloud-storage already installed


In [2]:
python -m pip install --upgrade pip

SyntaxError: invalid syntax (629497118.py, line 1)

In [3]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:0031m7.8 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
Note: you may need to restart the kernel to use updated packages.


In [8]:
# %% Cell 2: Set Up Authentication
import os
from pathlib import Path
import json

print("🔐 Setting up Google Cloud Authentication\n")
print("Choose your authentication method:\n")
print("1. Use existing gcloud CLI authentication (if you're already logged in)")
print("2. Use a service account JSON key file")
print("3. Set up new authentication")

# Check if already authenticated via gcloud
def check_gcloud_auth():
    """Check if gcloud is already authenticated"""
    try:
        from google.auth import default
        credentials, project = default()
        print(f"✅ Found existing authentication for project: {project}")
        return True, project
    except Exception as e:
        return False, None

is_authenticated, project_id = check_gcloud_auth()

if is_authenticated:
    print(f"\n✅ Already authenticated with project: {project_id}")
    USE_EXISTING_AUTH = True
else:
    print("\n❌ No existing authentication found")
    print("Please choose option 2 or 3 above")
    USE_EXISTING_AUTH = False

🔐 Setting up Google Cloud Authentication

Choose your authentication method:

1. Use existing gcloud CLI authentication (if you're already logged in)
2. Use a service account JSON key file
3. Set up new authentication
✅ Found existing authentication for project: quantum-sfa-phd

✅ Already authenticated with project: quantum-sfa-phd


In [2]:
# %% Cell 1: Import Required Libraries
from google.cloud import storage
from datetime import datetime
import json
import pickle
from pathlib import Path
import numpy as np

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [10]:
# %% Cell 2: Define Configuration (THIS WAS MISSING!)
"""
Define all your bucket names and project configuration
"""

# Your bucket configuration - DEFINE THIS FIRST
BUCKETS = {
    'results': 'quantum-sfa-results',
    'data': 'quantum-sfa-data',
    'archive': 'quantum-sfa-archive'
}

PROJECT_ID = 'quantum-brain-models'  # Update if your project ID is different

print("📦 Configured buckets:")
for purpose, bucket_name in BUCKETS.items():
    print(f"  - {purpose}: {bucket_name}")

📦 Configured buckets:
  - results: quantum-sfa-results
  - data: quantum-sfa-data
  - archive: quantum-sfa-archive


In [12]:
# %% Cell 3: Test Connection to Your Buckets
"""
Now test access to each bucket
"""

def test_bucket_access(bucket_name):
    """Test access to a specific bucket"""
    try:
        client = storage.Client(project=PROJECT_ID)
        bucket = client.bucket(bucket_name)
        
        # List first few items to verify access
        blobs = list(bucket.list_blobs(max_results=3))
        
        print(f"✅ Connected to: {bucket_name}")
        print(f"   Contains items (showing max 3):")
        
        if len(blobs) == 0:
            print(f"   - (bucket is empty)")
        else:
            for blob in blobs:
                size_kb = blob.size / 1024 if blob.size else 0
                print(f"   - {blob.name} ({size_kb:.1f} KB)")
        
        return bucket
        
    except Exception as e:
        print(f"❌ Failed to connect to {bucket_name}: {e}")
        return None

print("🔄 Testing connections to your buckets:\n")
connected_buckets = {}

for purpose, bucket_name in BUCKETS.items():
    print(f"\nTesting {purpose} bucket:")
    connected_buckets[purpose] = test_bucket_access(bucket_name)
    print("-" * 40)

if all(connected_buckets.values()):
    print("\n🎉 All buckets connected successfully!")
else:
    print("\n⚠️ Some buckets failed to connect. Check permissions.")

🔄 Testing connections to your buckets:


Testing results bucket:
✅ Connected to: quantum-sfa-results
   Contains items (showing max 3):
   - model4/Test1_PNC_Baseline_20250823_062949/data.json (0.1 KB)
   - model4/Test1_PNC_Baseline_20250823_063329/data.json (0.1 KB)
   - model4/Test1_PNC_Baseline_20250823_063555/data.json (0.1 KB)
----------------------------------------

Testing data bucket:
✅ Connected to: quantum-sfa-data
   Contains items (showing max 3):
   - (bucket is empty)
----------------------------------------

Testing archive bucket:
✅ Connected to: quantum-sfa-archive
   Contains items (showing max 3):
   - (bucket is empty)
----------------------------------------

🎉 All buckets connected successfully!


In [5]:
# %% Cell 4: Create the Storage Manager Class
"""
Unified manager for all your GCS operations
"""

class QuantumSFAStorage:
    """Manager for your Quantum-SFA storage buckets"""
    
    def __init__(self, project_id=None):
        self.project_id = project_id or PROJECT_ID
        self.client = storage.Client(project=self.project_id)
        
        # Connect to your three buckets
        self.results_bucket = self.client.bucket(BUCKETS['results'])
        self.data_bucket = self.client.bucket(BUCKETS['data'])
        self.archive_bucket = self.client.bucket(BUCKETS['archive'])
        
        print(f"✅ Storage Manager initialized for project: {self.project_id}")
        
    def save_model_results(self, model_name, results_dict, experiment_tag=None):
        """
        Save model results to the results bucket
        
        Args:
            model_name: e.g., 'model4_dynamic_nanoreactor'
            results_dict: Dictionary containing your results
            experiment_tag: Optional tag for this experiment
        """
        # Create organized path structure
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        date_folder = datetime.now().strftime('%Y_%m_%d')
        
        if experiment_tag:
            filename = f"{model_name}_{experiment_tag}_{timestamp}.json"
        else:
            filename = f"{model_name}_{timestamp}.json"
            
        # Path: results/YYYY_MM_DD/model_name/filename
        blob_path = f"results/{date_folder}/{model_name}/{filename}"
        
        # Upload to results bucket
        blob = self.results_bucket.blob(blob_path)
        blob.upload_from_string(
            json.dumps(results_dict, indent=2, default=str),
            content_type='application/json'
        )
        
        print(f"✅ Saved to: gs://{BUCKETS['results']}/{blob_path}")
        return blob_path
    
    def save_raw_data(self, data, data_type, description):
        """Save raw data (arrays, etc.) to the data bucket"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        blob_path = f"{data_type}/{description}_{timestamp}.pkl"
        
        blob = self.data_bucket.blob(blob_path)
        
        # Save as pickle for complex data structures
        blob.upload_from_string(pickle.dumps(data))
        
        print(f"✅ Raw data saved to: gs://{BUCKETS['data']}/{blob_path}")
        return blob_path
    
    def list_recent_results(self, model_name=None, limit=10):
        """List recent results from the results bucket"""
        prefix = "results/"
        if model_name:
            # Note: This won't perfectly filter by model name due to path structure
            # but will show recent results
            pass
            
        blobs = self.results_bucket.list_blobs(prefix=prefix, max_results=limit)
        
        print(f"\n📊 Recent Results (max {limit}):")
        print("-" * 60)
        
        results = []
        for blob in blobs:
            # Parse the path to get meaningful info
            parts = blob.name.split('/')
            if len(parts) >= 3:
                date = parts[1] if len(parts) > 1 else "unknown"
                model = parts[2] if len(parts) > 2 else "unknown"
                file = parts[-1] if len(parts) > 3 else blob.name
                
                print(f"  {date} | {model}")
                print(f"    └─ {file}")
                
                results.append({
                    'path': blob.name,
                    'created': blob.time_created,
                    'size': blob.size
                })
        
        return results
    
    def load_result(self, blob_path):
        """Load a specific result from storage"""
        blob = self.results_bucket.blob(blob_path)
        
        if blob.exists():
            content = blob.download_as_text()
            return json.loads(content)
        else:
            print(f"❌ Not found: {blob_path}")
            return None
    
    def today_results(self):
        """Show all results from today"""
        date_folder = datetime.now().strftime('%Y_%m_%d')
        prefix = f"results/{date_folder}/"
        
        blobs = self.results_bucket.list_blobs(prefix=prefix)
        
        print(f"\n📊 Today's Results ({date_folder}):")
        print("-" * 50)
        
        count = 0
        for blob in blobs:
            # Extract meaningful name from path
            parts = blob.name.split('/')
            if len(parts) >= 4:
                model = parts[2]
                filename = parts[3]
                print(f"  {model}: {filename}")
                count += 1
        
        if count == 0:
            print("  (No results yet today)")
        else:
            print(f"\nTotal: {count} results today")
        
        return count

# Initialize the storage manager
print("\n🔄 Initializing Storage Manager...")
storage_manager = QuantumSFAStorage()


🔄 Initializing Storage Manager...
✅ Storage Manager initialized for project: quantum-brain-models
